In [1]:
from mmdet.apis import init_detector, inference_detector
from mmdet.registry import VISUALIZERS
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# download model

In [2]:
!mim download mmdet --config rtmdet_tiny_8xb32-300e_coco --dest ./checkpoints

processing rtmdet_tiny_8xb32-300e_coco...
rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth exists in d:\ml\code\mm\mmdetection\checkpoints
Successfully dumped rtmdet_tiny_8xb32-300e_coco.py to d:\ml\code\mm\mmdetection\checkpoints


In [3]:
config_file = 'configs/rtmdet/rtmdet_tiny_8xb32-300e_coco.py'
checkpoint_file = 'checkpoints/rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth'
onnx_file = 'checkpoints/rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.onnx'

# model&infer

In [4]:
model = init_detector(config_file, checkpoint_file, device='cpu')
model

Loads checkpoint by local backend from path: checkpoints/rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: data_preprocessor.mean, data_preprocessor.std



RTMDet(
  (data_preprocessor): DetDataPreprocessor()
  (backbone): CSPNeXt(
    (stem): Sequential(
      (0): ConvModule(
        (conv): Conv2d(3, 12, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): _BatchNormXd(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activate): SiLU(inplace=True)
      )
      (1): ConvModule(
        (conv): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): _BatchNormXd(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activate): SiLU(inplace=True)
      )
      (2): ConvModule(
        (conv): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): _BatchNormXd(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activate): SiLU(inplace=True)
      )
    )
    (stage1): Sequential(
      (0): ConvModule(
        (conv): Conv2d(24, 48, kernel_size=(3, 3), stride=(2, 2), pad

# export onnx

In [5]:
x = torch.ones(1, 3, 640, 640)

In [11]:
torch.onnx.export(
    model=model,
    args=x,
    opset_version=17,
    f=onnx_file,
    input_names=["images"],
    output_names=["0_scores", "1_scores", "2_scores", "0_boxes", "1_boxes", "2_boxes"],
)

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [12]:
import onnx
from onnxsim import simplify

In [13]:
model_ = onnx.load(onnx_file)

In [14]:
model__, ok = simplify(model_)
print(ok)

True


In [15]:
onnx.save(model__, onnx_file)